In [44]:
import argparse
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.serialization import load_pem_public_key, load_pem_private_key
import os
import random

### 1. Генерация ключей гибридной системы

In [52]:
way = os.path.abspath('')
file_settings= {
    'initial_file': way + '\\' + 'initial_file.txt',
    'encrypted_file': way + '\\' + 'encrypted_file.txt',
    'decrypted_file': way + '\\' + 'decrypted_file.txt',
    'symmetric_key': way + '\\' + 'symmetric_key.txt',
    'public_key': way + '\\' + 'public_key.pem',
    'private_key': way + '\\' + 'private_key.pem',
}

keys = [i for i in range(40, 129, 8)]


генерация ключа для симметричного алгоритма

In [60]:
def gen_keys(file_settings: dict, keys: list):
    len_key = random.randint(0, len(keys) - 1)
    key = os.urandom(keys[len_key])
    print('Ключ симметричного шифрования:')
    print(type(key))
    print(key)
    
    assym_keys = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    private_key = assym_keys
    public_key = assym_keys.public_key()
    
    print('\nЗакрытый ключ ассимметричного шифрования:')
    print(type(private_key))
    print(private_key)
    print('\nОтрытый ключ ассимметричного шифрования:')
    print(type(public_key))
    print(public_key)
    
    c_key = public_key.encrypt(key, padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None))
    print('\nЗашифрованный ключ симметричного шифрования')
    print(c_key)
    # сериализация открытого ключа в файл
    with open(file_settings['public_key'], 'wb') as public_out:
            public_out.write(public_key.public_bytes(encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo))

    # сериализация закрытого ключа в файл
    with open(file_settings['private_key'], 'wb') as private_out:
            private_out.write(private_key.private_bytes(encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()))
            
    c_key = public_key.encrypt(key, padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None))
    with open(file_settings['symmetric_key'], 'wb') as key_file:
        key_file.write(c_key)

генерация пары ключей для acсимметричного алгоритма

In [61]:
gen_keys(file_settings, keys)

Ключ симметричного шифрования:
<class 'bytes'>
b'\xd8St\xdd\xf9\x0f\xf0\xd1[Y\xa4\x8a\xbd\x0e"8\xd0\x9ct\x13\xa2\t\xd5T\xcc\x83\xbf\xd3|\xa4\x9f\xd6\xf9\xe9\xed\'\x05\x88rMXy@\x15\xad\xddt\xd5'

Закрытый ключ ассимметричного шифрования:
<class 'cryptography.hazmat.backends.openssl.rsa._RSAPrivateKey'>

Отрытый ключ ассимметричного шифрования:
<class 'cryptography.hazmat.backends.openssl.rsa._RSAPublicKey'>

Зашифрованный ключ симметричного шифрования
b'/\xc4\x9cQ\xf2D\x00*\xefq5\xb1\xed6\xad\xb4R\xc5\xa9\x84\x9f\x95@\x02j\xf1y\xad\x0b\xe8\x0b\xf5\x9f\xe1l\x05\x1f\xbeK\x95\xaa\xd7;\xeet\xa9\n\t\x1c}cZ\xf2\xbf\xb9\x8c^H\xfb\xa3g&\xb5\x04U\x8b\xbf/\x1b\xdc\x7f\xdaV\x9d\xa2\xda~\xfc\x83\x80\x1d\x03\x86\xdd\xfb\x89yr\xffB2\xb0<\x19\xcc\xc9\xbf`%\xa4\x05\x8d\xb1s\xfc\xfcf\xeay1\xe9\x0c\xfbq\xf7\x17\xb7|W\x01\xaf\x83\xbee\xeb{\xa4\xd7\xc0k2\xed\x1dm\xc9a\xa7\xfa\xbcQ\x9e\x95`-\xd3\xa9cN\x90\xe61\r\xda\x0eE>\x8d\x15\xd56m\x05\x02p\x83o\x928!\xa5<A\x8a\xf2\xf4co\x16K_\x9am\xc6\x86\xfa\x85T!\x04

### 2. Шифрование данных гибридной системой